# Artificial Intelligence - Lab 3

In this laboratory we will explore a real classification problem involving high dimensional data (images). We will use the popular [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, which contains 70.000 images of handwritten digits, annotated with the corresponding labels (from 0 to 9).

For solving this problem we will implement a ***deep learning*** architecture, that is, a MLP with several hidden layers. We will also implement a convolutional neural network, specifically suited for image processing.

Finally, we will explore how to simulate a perceptual experimental task, by testing the model's robustness to noise injected in the visual stimuli.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import sklearn.metrics as metrics
from sklearn.neural_network import MLPClassifier

from torchvision.datasets import MNIST
from torchvision.transforms import Lambda

The MNIST is already split into training / test sets, so we don't need to use `train_test_split`. It is also already divided between input data (pixels) and output targets (digit classes).

In [ ]:
%%capture
mnist_tr = MNIST(root="../mnist", train=True, download=True)
mnist_te = MNIST(root="../mnist", train=False, download=True)

In [ ]:
mnist_tr, mnist_tr_labels = mnist_tr.data.numpy(), mnist_tr.targets.numpy()
mnist_te, mnist_te_labels = mnist_te.data.numpy(), mnist_te.targets.numpy()

The images are in a two dimensional format (28x28 matrices). However, the fully-connected MLP neural network requires one dimensional input vectors, thus the first step is to flatten the matrices with the function `reshape`, obtaining a vector of 784 elements.

Moreover, the images are saved in a conventional format, where each pixel can assume values between 0 and 255. Hence, the second step is to normalize such values to a 0-1 interval, simply by dividing by 255.

In [ ]:
x_tr = mnist_tr.reshape(60000, 28 * 28)
x_te = mnist_te.reshape(10000, 28 * 28)

In [ ]:
x_tr = x_tr / 255
x_te = x_te / 255

Let's visualize the first test pattern:

In [ ]:
_ = plt.imshow(x_te[0].reshape(28, 28), cmap="gray")

We create one MLP with two hidden layers, keeping the learning parameters to the default setting. Then, we can start the training.

In [ ]:
# with 10 iterations the convergence is already good enough and the algorithm
# takes approximately three minutes to run. For a better convergence it is
# possible to increase the number of iterations.

MLP = MLPClassifier(hidden_layer_sizes=(500, 500),
                    max_iter = 10,
                    random_state=42)

In [ ]:
MLP = MLP.fit(x_tr, mnist_tr_labels)

Now we can proceed by plotting the error curve, the mean accuracy and the confusion matrix.

In [ ]:
_ = plt.plot(range(MLP.n_iter_), MLP.loss_curve_)
_ = plt.xlabel("Epoch")
_ = plt.ylabel("Loss")
_ = plt.title("Loss minimization during training")

In [ ]:
MLP.score(x_te, mnist_te_labels)

In [ ]:
x_te_predictions = MLP.predict(x_te)
_ = metrics.ConfusionMatrixDisplay.from_predictions(mnist_te_labels, x_te_predictions)

## **EXERCISE: Study robustness to noise.**

*Tip #1: You can inject noise into the test images by adding a (Gaussian) random matrix to the input vectors using the np.random.normal function.*

*Tip #2: You can define the strength of the noise by changing the standard deviation of the Gaussian.*

*Tip #3: You can either directly add the noise to the entire dataset matrix, or you can add it to each individual image (in this case, you'll need a for loop).*

*Tip #4: Visualize the noisy images using the plt.imshow function, and compute the accuracy using the MLP.score function.*

## Convolutional Neural Network

Using a high-level Python framework (TensorFlow) we can even "easily" implement a more sophisticated Convolutional Neural Network!

In [ ]:
from tensorflow import keras

In [ ]:
model = keras.models.Sequential(
    [keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
     keras.layers.Flatten(),
     keras.layers.Dense(units=10, activation='softmax')]
)

model.compile(optimizer='adam',
              metrics=["accuracy"],
              loss='sparse_categorical_crossentropy')

Differently from fully-connected MLP, convolutional networks require 2D images as input:

In [ ]:
x_tr_conv = x_tr.reshape(-1, 28, 28, 1)
x_te_conv = x_te.reshape(-1, 28, 28, 1)

In [ ]:
loss_trend = model.fit(x_tr_conv, mnist_tr_labels, epochs=10)

In [ ]:
_ = plt.plot(range(10), loss_trend.history['loss'])
_ = plt.xlabel("Epoch")
_ = plt.ylabel("Loss")
_ = plt.title("Loss minimization during training")

In [ ]:
test_loss, test_accuracy = model.evaluate(x_te_conv, mnist_te_labels)

In [ ]:
_ = x_te_predictions_conv = model.predict(x_te_conv)
metrics.ConfusionMatrixDisplay.from_predictions(mnist_te_labels, x_te_predictions_conv.argmax(axis=1))

### Robustness to noise

In [ ]:
x_te_noisy_conv = x_te_noisy.reshape(-1, 28, 28, 1)
test_loss, test_accuracy = model.evaluate(x_te_noisy_conv, mnist_te_labels)